# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [23]:
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from utils import plot_confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

import scipy.stats as stats

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [20]:
dataset = pd.read_csv("../data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [24]:
clf = SGDClassifier(random_state=0)

clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

modelos = {'Precission score': precision_score,
          'Accuracy score': accuracy_score,
          'Recall score': recall_score,
          'F1 score': f1_score}

for model_name, model in modelos.items():
    train = model(y_train, y_train_pred)
    test = model(y_test, y_test_pred)
    print(f'Train {model_name}: {train:0.2}')
    print(f'Test {model_name}: {test:0.2}')

print('Matriz de confusion train:')
print(confusion_matrix(y_train, y_train_pred))
print('Matriz de confusion test:')
print(confusion_matrix(y_test, y_test_pred))

Train Precission score: 0.0
Test Precission score: 0.0
Train Accuracy score: 0.83
Test Accuracy score: 0.84
Train Recall score: 0.0
Test Recall score: 0.0
Train F1 score: 0.0
Test F1 score: 0.0
Matriz de confusion train:
[[1232    0]
 [ 251    0]]
Matriz de confusion test:
[[313   0]
 [ 58   0]]


### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [4]:
clf = SGDClassifier(random_state=0)

param_grid = {
    "loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "none"],
    "average": [True, False],
    #"l1_ratio": stats.uniform(0, 1),
}

cv = GridSearchCV(clf, param_grid, scoring='accuracy', cv=5)
cv.fit(X_train, y_train);

In [5]:
results = cv.cv_results_
df = pd.DataFrame(results)
df[['param_loss', 'param_alpha', 'mean_test_score', 'std_test_score', 'rank_test_score']]

,param_loss,param_alpha,mean_test_score,std_test_score,rank_test_score
0,hinge,0.0001,0.830749,0.001259,1
1,hinge,0.0001,0.830749,0.001259,1
2,hinge,0.0001,0.830749,0.001259,1
3,log,0.0001,0.830749,0.001259,1
4,log,0.0001,0.830749,0.001259,1
...,...,...,...,...,...
91,squared_hinge,0.1,0.823342,0.014412,69
92,squared_hinge,0.1,0.691582,0.261635,84
93,modified_huber,0.1,0.830749,0.001259,1
94,modified_huber,0.1,0.714251,0.234230,74


In [6]:
cv.best_estimator_

SGDClassifier(average=True, random_state=0)

In [7]:
cv.best_params_

{'alpha': 0.0001, 'average': True, 'loss': 'hinge', 'penalty': 'l2'}

In [8]:
best_model = cv.best_estimator_

best_model.fit(X_train, y_train)

y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

modelos = {'Precission score': precision_score,
          'Accuracy score': accuracy_score,
          'Recall score': recall_score,
          'F1 score': f1_score}

for model_name, model in modelos.items():
    train = model(y_train, y_train_pred)
    test = model(y_test, y_test_pred)
    print(f'Train {model_name}: {train:0.2}')
    print(f'Test {model_name}: {test:0.2}')

print('Matriz de confusion train:')
print(confusion_matrix(y_train, y_train_pred))
print('Matriz de confusion test:')
print(confusion_matrix(y_test, y_test_pred))

Train Precission score: 0.0
Test Precission score: 0.0
Train Accuracy score: 0.83
Test Accuracy score: 0.84
Train Recall score: 0.0
Test Recall score: 0.0
Train F1 score: 0.0
Test F1 score: 0.0
Matriz de confusion train:
[[1232    0]
 [ 251    0]]
Matriz de confusion test:
[[313   0]
 [ 58   0]]


/home/fligoonaut/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fligoonaut/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [10]:
clf = DecisionTreeClassifier(random_state=0)

clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

modelos = {'Precission score': precision_score,
          'Accuracy score': accuracy_score,
          'Recall score': recall_score,
          'F1 score': f1_score}

for model_name, model in modelos.items():
    train = model(y_train, y_train_pred)
    test = model(y_test, y_test_pred)
    print(f'Train {model_name}: {train:0.2}')
    print(f'Test {model_name}: {test:0.2}')

print('Matriz de confusion train:')
print(confusion_matrix(y_train, y_train_pred))
print('Matriz de confusion test:')
print(confusion_matrix(y_test, y_test_pred))

Train Precission score: 1.0
Test Precission score: 0.62
Train Accuracy score: 1.0
Test Accuracy score: 0.88
Train Recall score: 1.0
Test Recall score: 0.64
Train F1 score: 1.0
Test F1 score: 0.63
Matriz de confusion train:
[[1232    0]
 [   0  251]]
Matriz de confusion test:
[[290  23]
 [ 21  37]]


### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [11]:
clf = DecisionTreeClassifier(random_state=0)

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2,3],
    'min_samples_leaf': [1,2,3,4]
}

cv = GridSearchCV(clf, param_grid, scoring='accuracy', cv=5)
cv.fit(X_train, y_train);

In [13]:
results = cv.cv_results_
params = results['params']
mean = results['mean_test_score']
std = results['std_test_score']
rank = results['rank_test_score']

print("crit.\tdepth\t| mean\tstd\trank")
for p, m, s, r in zip(params, mean, std, rank):
    print(f"{p['criterion']}\t{p['max_depth']}\t| {m:0.2f}\t{s:0.2f}\t{r}")

crit.	depth	| mean	std	rank
gini	1	| 0.87	0.01	13
gini	1	| 0.87	0.01	13
gini	1	| 0.87	0.01	13
gini	1	| 0.87	0.01	13
gini	2	| 0.87	0.01	9
gini	2	| 0.87	0.01	9
gini	2	| 0.87	0.01	9
gini	2	| 0.87	0.01	9
gini	3	| 0.88	0.01	1
gini	3	| 0.88	0.01	1
gini	3	| 0.88	0.01	1
gini	3	| 0.88	0.01	1
entropy	1	| 0.87	0.01	13
entropy	1	| 0.87	0.01	13
entropy	1	| 0.87	0.01	13
entropy	1	| 0.87	0.01	13
entropy	2	| 0.87	0.01	21
entropy	2	| 0.87	0.01	21
entropy	2	| 0.87	0.01	21
entropy	2	| 0.87	0.01	21
entropy	3	| 0.87	0.01	5
entropy	3	| 0.87	0.01	5
entropy	3	| 0.87	0.01	5
entropy	3	| 0.87	0.01	5


In [14]:
best_model = cv.best_estimator_

best_model.fit(X_train, y_train)

y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

modelos = {'Precission score': precision_score,
          'Accuracy score': accuracy_score,
          'Recall score': recall_score,
          'F1 score': f1_score}

for model_name, model in modelos.items():
    train = model(y_train, y_train_pred)
    test = model(y_test, y_test_pred)
    print(f'Train {model_name}: {train:0.2}')
    print(f'Test {model_name}: {test:0.2}')

print('Matriz de confusion train:')
print(confusion_matrix(y_train, y_train_pred))
print('Matriz de confusion test:')
print(confusion_matrix(y_test, y_test_pred))

Train Precission score: 0.81
Test Precission score: 0.65
Train Accuracy score: 0.88
Test Accuracy score: 0.87
Train Recall score: 0.41
Test Recall score: 0.41
Train F1 score: 0.55
Test F1 score: 0.51
Matriz de confusion train:
[[1207   25]
 [ 147  104]]
Matriz de confusion test:
[[300  13]
 [ 34  24]]
